In [ ]:
!pip install python-binance
!pip install catboost
!pip install matplotlib --upgrade
!pip install mpl_finance
!pip install --upgrade mplfinance

In [ ]:
import pandas as pd
import numpy as np

import time
import dateparser
import pytz
import json

import math  

import datetime as dt
from datetime import datetime, timedelta

from threading import Thread

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates

from mpl_finance import candlestick_ochl



from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv

from tqdm import tqdm as tqdm

from binance.client import Client

import datetime as dtt

In [17]:
#partimos del csv V4
#V3_with_tarifas.to_csv('df_V4.csv')

In [ ]:
V4 = pd.read_csv('df_V4.csv')

In [ ]:
V4.dtypes

In [ ]:
V4['Open Time'] = pd.to_datetime(V4['Open Time'])

In [ ]:
print(V4)

In [ ]:
V4.describe()

In [ ]:
bonos = pd.read_csv('bonos_tesoro_EEUU_un_mes. OK.csv', sep=";" , decimal=",")

In [ ]:
bonos

In [ ]:
bonos.describe()

In [ ]:
bonos.dtypes

In [ ]:
bonos['Fecha'] = pd.to_datetime(bonos['Fecha'])

In [ ]:
bonos.dtypes

In [ ]:
print(bonos)

In [ ]:
V5 = pd.merge(left=V4, right=bonos, how='left', left_on='Open Time', right_on='Fecha')
V5.describe()

In [ ]:
V5.rename(columns={'Último':'Bonos_close'}, inplace= True)

In [ ]:
V5.describe()

In [ ]:
V5.describe()

In [ ]:
V5_1 = V5.drop(['Unnamed: 0','Unnamed: 0.1','Apertura', 'Máximo', 'Mínimo', '% var.', 'Fecha'], axis=1)

In [ ]:
V5_1.isna().sum()

In [ ]:
V5_1.columns.values

In [ ]:
V5_1.tipoint

In [ ]:
print(V5_1)

In [ ]:
V5_1.fillna(method='ffill', inplace=True)

In [ ]:
def evaluate_detection(series, true_indices, detected_indices):
    
    # calculate metrics
    tp = list(set(detected_outliers).intersection(set(true_indices)))
    fp = list(set(detected_outliers).difference(set(true_indices)))
    fn = list(set(true_indices).difference(set(detected_outliers)))
    perc_detected = 100 * len(tp) / len(true_indices)
    
    # create the plot
    fix, ax = plt.subplots(2, 1,figsize=(10,10))
    
    ax[0].plot(np.arange(len(series)), series);
    ax[0].scatter(true_indices, series[true_indices], c='g', label='true outlier')
    ax[0].set_title('Original series')
    ax[0].legend()
    
    ax[1].plot(np.arange(len(series)), series);
    ax[1].scatter(tp, series[tp], c='g', label='true positive')
    ax[1].scatter(fp, series[fp], c='r', label='false positive')
    ax[1].scatter(fn, series[fn], c='k', label='false negative')
    ax[1].set_title('Algorithm results')
    ax[1].legend()
    
    # print out summary
    print('-' * 25 + ' Summary ' + '-' * 25)
    print(f'Outliers in the series: {len(true_indices)}')
    print(f'Identified outliers: {len(detected_indices)}')
    print(f'Correctly detected outliers: {len(tp)} ({perc_detected:.2f}% of all outliers).')
    print('-' * 59)
    
    return tp, fp, fn

In [ ]:
def hampel_filter_forloop(input_series, window_size, n_sigmas=3):
    
    n = len(input_series)
    new_series = input_series.copy()
    k = 1.4826 # scale factor for Gaussian distribution
    
    indices = []
    
    # possibly use np.nanmedian 
    for i in range((window_size),(n - window_size)):
        x0 = np.median(input_series[(i - window_size):(i + window_size)])
        S0 = k * np.median(np.abs(input_series[(i - window_size):(i + window_size)] - x0))
        if (np.abs(input_series[i] - x0) > n_sigmas * S0):
            new_series[i] = x0
            indices.append(i)
    
    return new_series, indices

In [ ]:
res, detected_outliers = hampel_filter_forloop(V5_1['Bonos_close'],11)



In [ ]:
detected_outliers

In [ ]:
outlier_ind = detected_outliers

In [ ]:
tp, fp, fn = evaluate_detection(V5_1['Bonos_close'], outlier_ind, detected_outliers)


In [ ]:
plt.plot(np.arange(len(res)), res);
plt.scatter(outlier_ind, V5_1['Bonos_close'][outlier_ind], c='g', label='true outlier')
plt.title('Cleaned series (without detected outliers)')
plt.legend();

In [ ]:
V5_1['Bonos_close'] = res

In [ ]:
V5_1.Bonos_close.plot()
plt.plot()

In [ ]:
V5_1.dtypes

In [ ]:
V5_1.tail(400)

In [ ]:
V5_1.to_csv('V5.csv')

In [ ]:
V5_1.describe()

In [ ]:
gas_natural = pd.read_csv('gas_natural.ok.csv', sep=";" , decimal=",")


In [ ]:
gas_natural.describe()

In [ ]:
gas_natural.head()

In [ ]:
gas_natural_dep = gas_natural.drop(['Apertura','Máximo','Mínimo','Vol.','% var.'], axis=1)

In [ ]:
gas_natural_dep.isna().sum()

In [ ]:
gas_natural_dep.dtypes

In [ ]:
gas_natural_dep['Fecha'] = pd.to_datetime(gas_natural_dep['Fecha'])

In [ ]:
gas_natural_dep.rename(columns={'Fecha':'Open Time'}, inplace=True)

In [ ]:
V5_1_with_gas = V5_1.merge(gas_natural_dep[["Último", "Open Time"]], how='left', on="Open Time")


In [ ]:
print(V5_1_with_gas)

In [ ]:
V5_1_with_gas.isna().sum()

In [ ]:
V5_1_with_gas.drop(['Apertura', 'Máximo', 'Mínimo', '% var.', 'Fecha'], axis=1)

In [ ]:
V5_with_gas.rename(columns={'Último':'Gas_natural'}, inplace=True)

In [ ]:
V5_with_gas.drop(['Apertura', 'Máximo', 'Mínimo', '% var.', 'Fecha','Unnamed: 0','Unnamed: 0.1'], axis=1)

In [ ]:
print(V5_with_gas)

In [ ]:
V5_with_gas.isna().sum()

#dejamos en stand by la columna del gas natural. puesto que faltan 601 valores. valoramos a futuro, cuando estén todas las variables, rellenar con datos sintéticos 

In [ ]:
V5_with_gas.tail(400)

In [ ]:
indice_mercado_shanghai = pd.read_csv('indice_mercado_shanghai.csv',  sep=";" , decimal=",")




In [ ]:
print(indice_mercado_shanghai)

In [ ]:
indice_mercado_shanghai.describe()

In [ ]:
ind_shanghai = indice_mercado_shanghai.drop(['Apertura', 'Máximo', 'Mínimo', '% var.', 'Vol.'], axis=1)

In [ ]:
index_shanghai = ind_shanghai.rename(columns={'index_shangay':'index_shanghai'}, inplace=True)

In [ ]:
print(ind_shanghai)

In [ ]:
ind_shanghai = indice_mercado_shanghai.drop(['Apertura', 'Máximo', 'Mínimo', '% var.', 'Vol.'], axis=1)

In [ ]:
ind_shanghai.rename(columns={'Fecha':'Open Time'}, inplace=True)

In [ ]:
V5_with_gas.dtypes

In [ ]:
ind_shanghai.dtypes

In [ ]:
ind_shanghai['Open Time'] = pd.to_datetime(ind_shanghai['Open Time'])

In [ ]:
V6 = V5_with_gas.merge(ind_shanghai[["index_shanghai", "Open Time"]], how="left", on="Open Time")


In [ ]:
print(V6)

In [ ]:
V6.isna().sum()
#DEJAMOS EN STAND BY LAS VARIABLES DE GAS NATURAL Y DE INDEX SHANGAI. TRAS EL ACOPLE DE TODAS LAS VARIABLES, VALORAMOS QUÉ RELLENAMOS CON SINTÉTICAS. 


In [ ]:
precios_petroleo = pd.read_csv('precios_petroleo.csv ',  sep=";" , decimal=",")





In [ ]:
print(precios_petroleo)

In [ ]:
precios_petroleo.describe()

In [ ]:
pr_petroleo = precios_petroleo.drop(['Apertura', 'Máximo', 'Mínimo', '% var.', 'Vol.'], axis=1)

In [ ]:
pr_petroleo.describe()

In [ ]:
print(pr_petroleo)

In [ ]:
pr_petroleo.rename(columns={'Fecha':'Open Time','Último':'Ind_Petroleo'}, inplace=True)

In [ ]:
pr_petroleo.dtypes

In [ ]:
pr_petroleo['Open Time'] = pd.to_datetime(pr_petroleo['Open Time'])

In [ ]:
V7 = V6.merge(pr_petroleo[["Ind_Petroleo", "Open Time"]], how="left", on="Open Time")

In [ ]:
print(V7)

In [ ]:
V7.isna().sum()

In [ ]:
euro = pd.read_csv('Datos historicos EUR_USD.OK.csv',  sep=";" , decimal=",")

In [ ]:
print(euro)

In [ ]:
euro.describe()

In [ ]:
euro.dtypes

In [ ]:
euro['Fecha'] = pd.to_datetime(euro['Fecha'])

In [ ]:
euro.rename(columns={'Último':'Euro'}, inplace=True)

In [ ]:
euro_1 = euro.drop(['Apertura', 'Máximo', 'Mínimo', '% var.'], axis=1)

In [ ]:
euro_1.dtypes

In [ ]:
euro_1.rename(columns={'Fecha':'Open Time'}, inplace=True)

In [ ]:
print(euro_1)

In [ ]:
V8 = V7.merge(euro_1[["Euro", "Open Time"]], how="left", on="Open Time")

In [ ]:
print(V8)

In [ ]:
V8.isna().sum()

In [ ]:
yen = pd.read_csv('Datos historicos USD_JPY.OK.csv',  sep=";" , decimal=",")

In [ ]:
print(yen)

In [ ]:
yen.describe()

In [ ]:
yen.dtypes

In [ ]:
yen['Fecha'] = pd.to_datetime(yen['Fecha'])

In [ ]:
yen.rename(columns={'Fecha':'Open Time','Último':'Yen'}, inplace=True)

In [ ]:
yen_1 = yen.drop(['Apertura', 'Máximo', 'Mínimo', '% var.'], axis=1)

In [ ]:
yen_1.describe()

In [ ]:
print(yen_1)

In [ ]:
V9 = V8.merge(yen_1[["Yen", "Open Time"]], how="left", on="Open Time")

In [ ]:
print(V9)

In [ ]:
V9.isna().sum()

In [ ]:
V9.to_csv('V9.csv')

In [ ]:
yuan = pd.read_csv('yuan.ok.csv',  sep=";" , decimal=",")

In [ ]:
print(yuan)

In [ ]:
yuan.describe()

In [ ]:
yuan.dtypes

In [ ]:
yuan['Fecha'] = pd.to_datetime(yuan['Fecha'])

In [ ]:
yuan.rename(columns={'Fecha':'Open Time','Último':'Yen'}, inplace=True)

In [ ]:
yuan_1 = yuan.drop(['Apertura', 'Máximo', 'Mínimo', '% var.'], axis=1)

In [ ]:
yuan_1.describe()

In [ ]:
print(yuan_1)

In [ ]:
V10 = V9.merge(yuan_1[["Yen", "Open Time"]], how="left", on="Open Time")

In [ ]:
print(V10)

In [ ]:
V10.isna().sum()

In [ ]:
V10.to_csv('V10.csv')